# 01 - Data Cleaning and Merge
This notebook standardizes CMHC (rents), BoC (interest), and StatCan (employment) and exports `data/merged_data.csv`.

In [ ]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path('data')
cmhc_path = DATA_DIR / 'cmhc_rental_ottawa.xlsx'  # supports .xlsx or .csv
boc_path = DATA_DIR / 'boc_interest_rates.csv'
statcan_path = DATA_DIR / 'statcan_employment_ottawa.csv'
out_path = DATA_DIR / 'merged_data.csv'

def read_any(p: Path) -> pd.DataFrame:
    s = p.suffix.lower()
    if s == '.csv':
        return pd.read_csv(p)
    if s in {'.xlsx', '.xls'}:
        return pd.read_excel(p)
    return pd.read_csv(p)

cmhc = read_any(cmhc_path)
boc = read_any(boc_path)
stat = read_any(statcan_path)
cmhc.head(), boc.head(), stat.head()

## Rename columns to standard schema
Update the mappings below if your column names differ.

In [ ]:
# EDIT THESE if your columns differ
cmhc_map = { 'Year': 'Year', 'Avg_Rent': 'Avg_Rent', 'Vacancy_Rate': 'Vacancy_Rate' }
boc_map = { 'Year': 'Year', 'Interest_Rate': 'Interest_Rate' }
stat_map = { 'Year': 'Year', 'Employment_Rate': 'Employment_Rate', 'Unemployment_Rate': 'Unemployment_Rate' }

def standardize(df, mapping, required):
    missing = [src for src in required if src not in df.columns]
    if missing:
        raise ValueError(f'Missing columns: {missing}. Found: {list(df.columns)}')
    use = [c for c in mapping.keys() if c in df.columns]
    return df[use].rename(columns=mapping)

cmhc_std = standardize(cmhc, cmhc_map, ['Year', 'Avg_Rent'])
boc_std = standardize(boc, boc_map, ['Year', 'Interest_Rate'])
stat_std = standardize(stat, stat_map, ['Year', 'Employment_Rate'])
cmhc_std.head(), boc_std.head(), stat_std.head()

## Ensure types and merge by Year

In [ ]:
for df in (cmhc_std, boc_std, stat_std):
    df['Year'] = pd.to_datetime(df['Year'], errors='coerce').dt.year
    for c in df.columns:
        if c != 'Year':
            df[c] = pd.to_numeric(df[c], errors='coerce')

merged = (cmhc_std.merge(boc_std, on='Year', how='left')
                   .merge(stat_std, on='Year', how='left')
                   .sort_values('Year').reset_index(drop=True))

merged['Rent_YoY_Pct'] = merged['Avg_Rent'].pct_change() * 100
merged['Employment_YoY_Pp'] = merged['Employment_Rate'].diff()
merged['Employment_YoY_Pct'] = merged['Employment_Rate'].pct_change() * 100

merged.to_csv(out_path, index=False)
out_path, merged.shape, merged.head()